In [5]:
import numpy as np
import pandas as pd
def inventory_KPI(df, method_name, c_ratio):
    '''
    Calculate KPIs for sales evaluation using quantity suggested by method
    Parameters:
    - df: pandas DataFrame indexed by date, must have columns: '(Method Name)_Q' (predicted) and 'Verkauf_MBR' (ground truth)
    - method_name: string name of the method being evaluated

    Output:
    OOS rate, alpha service level, beta rate, lost sales, and pinball_loss over all rows
    '''
    def pinball_loss_calculator(row):
        if row['Verkauf_MBR'] >= row[method_name+'_Q']: return c_ratio*(row['Verkauf_MBR']-row[method_name+'_Q'])
        else: return (1-c_ratio)*(-row['Verkauf_MBR']+row[method_name+'_Q'])

    df['avg_oos_rate'] = df['Verkauf_MBR']>df[method_name+"_Q"]
    df['avg_beta_fill_rate'] = df.apply(lambda row: min(row['Verkauf_MBR'], row[method_name+"_Q"])/row['Verkauf_MBR'] if row['Verkauf_MBR']!=0 else None, axis=1)
    df['lost_sales'] = df.apply(lambda row: np.abs(row['Verkauf_MBR']-row[method_name+'_Q']), axis=1)
    df['pinball_loss'] = df.apply(pinball_loss_calculator, axis=1)

    OOS_rate = df['avg_oos_rate'].mean()
    alpha_level = 1 - OOS_rate
    beta_rate = df['avg_beta_fill_rate'].mean()
    lost_sales = df['lost_sales'].sum()
    pinball_loss = df['pinball_loss'].mean()
    
    return {
        method_name + '_avg_oos_rate' : OOS_rate,
        method_name + '_avg_beta_fill_rate': beta_rate,
        method_name + '_avg_alpha_service_level' : alpha_level,
        method_name + '_lost_sales': lost_sales,
        method_name + '_pinball_loss': pinball_loss
    }

def inventory_KPI_POS(dataframe, methods, magazines, c_ratio=0.9):
    '''
    dataframe: df containing all inventory suggestions and original data columns
    Calculate uncensoring KPIs grouped by POS.
    Returns dataframe with KPI of each method per magazine and POS
    '''
    df = dataframe.copy()
    df=df.dropna(subset=['Verkauf', 'Bezug'])
    df = df[df['Heftjahr']==2024]
    
    df_KPIs = pd.DataFrame() #initialize final output KPI dataframe
    for letter in magazines:
        print('-'*10+ f'Magazine {letter} KPIs' + '-'*10)
        df_magazine = df[df['Magazine']==letter]
        df_magazine_KPIs = None #initialize KPI dataframe for this magazine

        for method in methods:
            method_KPIs = [] #initialize KPIs for this magazine and method
            
            for pos, group in df_magazine.groupby('EHASTRA_EH_NUMMER'):
                pos_result = inventory_KPI(group, method, c_ratio)

                #catch None output
                if pos_result is not None:
                    pos_result['EHASTRA_EH_NUMMER'] = pos
                    method_KPIs.append(pos_result)
            df_method_KPIs = pd.DataFrame(method_KPIs)
            if len(df_method_KPIs)==0: continue
            df_method_KPIs['Magazine']=letter

            print('-'*10)
            kpi_cols = [c for c in df_method_KPIs.columns if c not in ['Magazine', 'EHASTRA_EH_NUMMER']]
            for col in kpi_cols:
                print(f'{col}: {df_method_KPIs[col].mean():.3f}')

            if df_magazine_KPIs is None: df_magazine_KPIs = df_method_KPIs
            else:
                new_cols = [c for c in df_method_KPIs if c not in df_magazine_KPIs or c in ['EHASTRA_EH_NUMMER']]
                df_magazine_KPIs = pd.merge(df_magazine_KPIs, df_method_KPIs[new_cols], how = 'outer', on = ['EHASTRA_EH_NUMMER'])
        df_KPIs = pd.concat([df_KPIs, df_magazine_KPIs])

    #move magazine and pos columns to the front
    df_KPIs.set_index(['Magazine', 'EHASTRA_EH_NUMMER'], inplace=True)
    df_KPIs.reset_index(inplace=True)
    return df_KPIs

In [6]:
df = pd.read_csv('Inventory_results.csv')
uncensoring_methods = ['Baseline', 'N1', 'N2', 'N3', 'EM', 'PD', 'Nahmias', 'Conrad', 'Bayesian']
magazine_letters = [letter for letter in 'ABCDEFGHI']
df_KPIs = inventory_KPI_POS(df, uncensoring_methods, magazine_letters, c_ratio=0.9)
df_KPIs.to_csv('Inventory_KPIs.csv', index=False) #adjust accordingly

----------Magazine A KPIs----------
----------
Baseline_avg_oos_rate: 0.039
Baseline_avg_beta_fill_rate: 0.985
Baseline_avg_alpha_service_level: 0.961
Baseline_lost_sales: 38.857
Baseline_pinball_loss: 0.187
----------
N1_avg_oos_rate: 0.049
N1_avg_beta_fill_rate: 0.981
N1_avg_alpha_service_level: 0.951
N1_lost_sales: 36.714
N1_pinball_loss: 0.182
----------
N2_avg_oos_rate: 0.049
N2_avg_beta_fill_rate: 0.981
N2_avg_alpha_service_level: 0.951
N2_lost_sales: 36.714
N2_pinball_loss: 0.182
----------
N3_avg_oos_rate: 0.039
N3_avg_beta_fill_rate: 0.985
N3_avg_alpha_service_level: 0.961
N3_lost_sales: 38.857
N3_pinball_loss: 0.187
----------
EM_avg_oos_rate: 0.039
EM_avg_beta_fill_rate: 0.985
EM_avg_alpha_service_level: 0.961
EM_lost_sales: 43.000
EM_pinball_loss: 0.201
----------
PD_avg_oos_rate: 0.039
PD_avg_beta_fill_rate: 0.985
PD_avg_alpha_service_level: 0.961
PD_lost_sales: 38.857
PD_pinball_loss: 0.187
----------
Nahmias_avg_oos_rate: 0.039
Nahmias_avg_beta_fill_rate: 0.985
Nahmias_a